In [ ]:
!pip install pandas
!pip install requests

In [ ]:
import pandas as pd
import os
import requests
import time
import re

In [ ]:
import re

def extract_year_period_week(filename):
    """Extracts year, period, and week from a filename like '2024_p1_w1.csv'.

    Args:
        filename: The name of the file.

    Returns:
        A tuple containing the year, period, and week as integers.
    """
    match = re.search(r"(\d+)_p(\d+)_w(\d+)", filename)
    if match:
        year = int(match.group(1))
        period = int(match.group(2))
        week = int(match.group(3))
        return year, period, week
    else:
        return None, None, None  # Handle cases where the pattern is not found

In [ ]:
import pandas as pd

for filename in ['2024_p7_w3.csv']:
    df = pd.read_csv(filename, dtype={'GROWER': str, 'TERRAIN': str})
    year, period, week = extract_year_period_week(filename)

    if year is not None and period is not None and week is not None:
        df['year'] = year  # Add 'Year' column
        df['period'] = period  # Add 'Period' column
        df['week'] = week  # Add 'Week' column
    else:
        print(f"Warning: Could not extract year, period, and week from {filename}")

    # Process the DataFrame further...

In [ ]:
def rename_csv_files(directory):
  """Renames CSV files in a directory from 'p{period}_w{week}.csv' to 'processed_p{period}_w{week}.csv'.

  Args:
    directory: The directory containing the CSV files.
  """
  for filename in os.listdir(directory):
    if filename.endswith(".csv") and filename.startswith("p"):
      base_name = os.path.splitext(filename)[0]  # Get filename without extension
      period, week = base_name[1:].split("_")  # Extract period and week
      new_filename = f"processed_{period}_{week}.csv"
      os.rename(os.path.join(directory, filename), os.path.join(directory, new_filename))
      print(f"Renamed '{filename}' to '{new_filename}'")

# This part is a merged code to put all data to a pandas dataframe and load it to RAM

In [ ]:
import os
import pandas as pd
import re

def extract_year_period_week(filename):
    """Extracts year, period, and week from a filename like '2024_p1_w1.csv'.

    Args:
        filename: The name of the file.

    Returns:
        A tuple containing the year, period, and week as integers.
    """
    match = re.search(r"(\d+)_p(\d+)_w(\d+)", filename)
    if match:
        year = int(match.group(1))
        period = int(match.group(2))
        week = int(match.group(3))
        return year, period, week
    else:
        return None, None, None  # Handle cases where the pattern is not found

def process_csv_files(directory):
    """Processes all CSV files in a directory, ingests data into a pandas DataFrame, and melts the DataFrame.

    Args:
        directory: The directory containing the CSV files.

    Returns:
        A pandas DataFrame containing the melted data from all CSV files.
    """
    all_data = []  # List to store data from all files

    for filename in os.listdir(directory):
        if filename.endswith(".csv") and filename.startswith("20"):  # Process only CSV files starting with "20"
            filepath = os.path.join(directory, filename)
            df = pd.read_csv(filepath, dtype={'GROWER': str, 'TERRAIN': str})
            
            year, period, week = extract_year_period_week(filename)
            if year is not None and period is not None and week is not None:
                df['year'] = year  # Add 'Year' column
                df['period'] = period  # Add 'Period' column
                df['week'] = week  # Add 'Week' column
            else:
                print(f"Warning: Could not extract year, period, and week from {filename}")

            # Melt the DataFrame
            melted_df = pd.melt(df,
                               id_vars=['GROWER', 'TERRAIN', 'BAGGING', 'STEMS', 'KLS.A', 'BXS.A', 'KLS.B', 'BXS.B', 'startdate', 'enddate', 'year', 'period', 'week'],
                               value_vars=['WK_10', 'WK_11', 'WK_12', 'WK_13', 'WK_14', 'WK_15', 'WK_16', 'WK_17'],
                               var_name='Week_Name',
                               value_name='stem_harvest_week_count')

            all_data.append(melted_df)  # Append melted data to the list

    # Concatenate all data into a single DataFrame
    final_df = pd.concat(all_data, ignore_index=True)
    return final_df

# Example usage:
directory_path = './data'  # Replace with the actual directory path
final_df = process_csv_files(directory_path)
final_df.head()
# Now you have the final_df containing data from all CSV files in the directory

In [ ]:
import json
import requests
import pandas as pd
from tqdm.auto import tqdm 
# Replace with your Strapi API endpoint for creating entries
strapi_api_endpoint = "http://127.0.0.1:1337/api/harvests"

def format_date(date_str):
  """Formats a date string from 'dd-mm-yyyy' to 'yyyy-MM-dd'.

  Args:
    date_str: The date string in 'dd-mm-yyyy' format.

  Returns:
    The formatted date string in 'yyyy-MM-dd' format.
  """
  return pd.to_datetime(date_str, format='%d-%m-%Y').strftime('%Y-%m-%d')


# Iterate through the rows of the final_df
for index, row in tqdm(final_df.iterrows(), total=final_df.shape[0]):  
  # Create a dictionary with the data for your Strapi content type
  data = { "data":{
      "grower": row["GROWER"],
      "terrainid": row["TERRAIN"],
      "bagging": row["BAGGING"],
      "stems": row["STEMS"],
      "class_type_a": row["KLS.A"],
      "box_type_a": row["BXS.A"],
      "class_type_b": row["KLS.B"],
      "box_type_b": row["BXS.B"],
      "stem_harvest_week": row["Week_Name"],
      "stem_harvest_count": row["stem_harvest_week_count"],
      "startdate": format_date(row["startdate"]),
      "enddate": format_date(row["enddate"]),
      "period": int(row["period"]),
      "week": int(row["week"]),
      "year": str(row["year"]),
      # Add other fields as needed
  }}  
 
  try:
   # Convert data to JSON string with double quotes
    json_data = json.dumps(data)  
    print(f"Sending Data : {json_data}")
    # Make POST request with the JSON string
    response = requests.post(strapi_api_endpoint, data=json_data, headers={'Content-Type': 'application/json'})

    response.raise_for_status()
    print(f"Successfully created entry for row {index}")    
  except requests.exceptions.RequestException as e:
    print(f"Error creating entry for row {index}: {e}")
  time.sleep(.5)  # Wait for 1 second before processing the next row

# We then save to csv the processed data

In [20]:
final_df.to_csv("2024_production_data.csv", index=False)